In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import ternary # ternary plot has a bug
import plotly.express as px
import h5py
import matplotlib as mpl

# import geopandas as gpd

from matplotlib.patches import Circle, Wedge, Polygon

import cmocean # Beautiful colormaps for oceanography
import cmocean.cm as cmo
import os
import matplotlib.patheffects as path_effects


from netCDF4 import Dataset as NetCDFFile 

from obspy.imaging.beachball import beach

# for beachball - there are some error with using obspy
from pyrocko.plot import beachball
from pyrocko import moment_tensor as ptm

import cartopy.crs as ccrs
import cartopy.mpl.geoaxes

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
from pathlib import Path
%matplotlib inline

import harmonica as hm

from obspy.core import UTCDateTime

from matplotlib.patches import Polygon

import seaborn as sns
sns.set()
sns.set_style(style='white') 
sns.set_style("ticks", {"xtick.major.size": 0.5, "ytick.major.size": 0.5})

# set the font globally
plt.rcParams.update({'font.family':'Helvetica'})

In [19]:
# pip install cmocean

In [20]:
# earthquakes
data = pd.read_csv("/Users/mohd0001/Research/PhD_Monash/Paper2/time-series/data/SUeq_data.csv",  header = 'infer')

data.sort_values(by = 'MW', ascending = False, inplace = True)

data['DATE'] = pd.to_datetime(data['DATE']) # change dtype to datetime format
data['DATE'] = data['DATE'].dt.strftime('%Y-%m-%d') # change dtype to object

# data['DATE'] and data['TIME'] should be object (str) to perform + operator

data_dummy = np.array(data['DATE'] + ' ' + data['TIME'])

data_ev_time = []

for val in data_dummy:
    data_ev_time.append(UTCDateTime(val))

    
ref_date = data_ev_time[0]

data_ev_time = np.array(data_ev_time)

date = []

for val in data_ev_time:
    date.append(val - ref_date)
    
date = np.array(date) # this is in seconds

data['YEARS'] = date/31536000

data = data.sort_values(by ='YEARS', ascending=True) # this is an important step
# data = data[(data['DATE'] > '01-01-2003') & (data['DATE'] < '26-11-2004')]

In [21]:
data = data[(data['CLASS'] == 'UPPER')
            & (data['TYPE'] == 'SS') &
            (data['P(km)'] > 200)]

data = data[(data['LAT'] > -6) & (data['LAT'] < 6)]

In [22]:
data.columns

Index(['LON', 'LAT', 'DEPTH', 'DATE', 'TIME', 'EVENT_ID', 'DEPFIX', 'DEPQUAL',
       'EX', 'MO', 'MW', 'EX.1', 'MRR', 'MTT', 'MPP', 'MRT', 'MTP', 'MPR',
       'STRIKE', 'DIP', 'RAKE', 'STRIKE.1', 'DIP.1', 'RAKE.1', 'EX.2', 'T_VAL',
       'T_PL', 'T_AZM', 'P_VAL', 'P_PL', 'P_AZM', 'N_VAL', 'N_PL', 'N_AZM',
       'SLAB_DIP', 'CLASS', 'P(km)', 'Q(km)', 'R(deg)', 'S(deg)', 'ALONG',
       'PROF', 'TTSUM', 'SINSQ_T', 'SINSQ_SS', 'SINSQ_N', 'TYPE', 'CORR_T_PL',
       'CORR_N_PL', 'CORR_P_PL', 'SINSQ_T_2', 'SINSQ_SS_2', 'SINSQ_N_2',
       'TYPE_2', 'CLASS1', 'TYPE1', 'Unnamed: 56', 'CLASS.1', 'Unnamed: 58',
       'Unnamed: 59', 'YEARS'],
      dtype='object')

In [23]:
data.DEPTH.max()

73.3

In [24]:
# data_mega = data[(data['MW'] >=8.5) & (data['YEARS'] < 3)]

In [25]:
print('Min depth is:', data['DEPTH'].max())

Min depth is: 73.3


In [26]:
## crustal age

nc = NetCDFFile("/Users/mohd0001/PhD_Monash/Paper1/Data/Mueller2008.nc")

# print(type(nc)) # print the type of the data 
# print(nc.variables.keys()) # check the variables in the data

lat_age = nc.variables['lat'][:]
lon_age = nc.variables['lon'][:]
z_age = nc.variables['z'][:]

# creating meshgrid from our list of spatial coordinates
lon_age, lat_age = np.meshgrid(lon_age,lat_age)

lat_age = nc.variables['lat'][:]
lon_age = nc.variables['lon'][:]
z_age = nc.variables['z'][:]

# creating meshgrid from our list of spatial coordinates
lon_age, lat_age = np.meshgrid(lon_age,lat_age)

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/mohd0001/PhD_Monash/Paper1/Data/Mueller2008.nc'

In [ ]:
# topo
nc = NetCDFFile('topo_sumatra.nc')
# nc = NetCDFFile("/Users/mohd0001/PhD_Monash/Paper1/Data/topo/TopographyWorld.nc")
# print(nc)
lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]
z = nc.variables['z'][:]
# creating meshgrid from our list of spatial coordinates
lon, lat = np.meshgrid(lon,lat)

In [ ]:
z.min()

In [ ]:
# trench
trench_xy = pd.read_csv("/Users/mohd0001/PhD_Monash/Paper2/time-series/data/plate_boundary.csv")
trench_xy.columns = ['LON', 'LAT'] # column names added

andaman_trench = trench_xy[trench_xy['LAT'] > 6]
sumatra_trench = trench_xy[(trench_xy['LAT'] < 6) & (trench_xy['LAT'] > -1)]
java_trench = trench_xy[(trench_xy['LAT'] < -1)]
# trench_xy.head()

In [ ]:
#slab 2.0
slab = NetCDFFile("/Users/mohd0001/PhD_Monash/Paper2/time-series/data/SUslab_depth.grd")
# print(type(slab)) # print the type of the data 
# print(slab.variables.keys()) # print the variables in the data

lon_slab = slab.variables['x'][:]
lat_slab = slab.variables['y'][:]
depth_slab = slab.variables['z'][:]

# creating meshgrid from our list of spatial coordinates
lon_slab, lat_slab = np.meshgrid(lon_slab, lat_slab)

In [ ]:
# rupture outline
rup_2004 = pd.read_csv("/Users/mohd0001/PhD_Monash/Paper2/time-series/data/2004_SUM_rup.csv")
# rup_2004.coulmns = ['LON', 'LAT']

rup_2005 = pd.read_csv("/Users/mohd0001/PhD_Monash/Paper2/time-series/data/2005_SUM_rup.csv")
# rup_2005.coulmns = ['LON', 'LAT']

rup_2007 = pd.read_csv("/Users/mohd0001/PhD_Monash/Paper2/time-series/data/2007_SUM_rup.csv")
# rup_2007.coulmns = ['LON', 'LAT']

# sum_rup.coulmns = ['LON', 'LAT']
# rup_2005.head()

In [ ]:
# data = data[(data['P(km)'] > 0)]
# data_dw = data[(data['DEPTH'] > 0) &
#                  (data['CLASS'] == 'DGOING') &
#                  (data['YEARS'] > 0)]

data_dw = data[(data['DEPTH'] > 0)&
                 (data['CLASS'] == 'UPPER')]

## sort by events thrust, normal, strike-slip
dataT = data_dw[(data_dw['TYPE'] == 'THRUST')]
dataN = data_dw[(data_dw['TYPE'] == 'NORM')]
dataS = data_dw[(data_dw['TYPE'] == 'SS')]

In [ ]:
print('Min depth is:', data_dw['DEPTH'].min())
print('Max depth is:', data_dw['DEPTH'].max())
# print(data_dw['P(km)'].min())
print('Min MW is:', data_dw['MW'].min())
print('Max MW is:', data_dw['MW'].max())
print('Max Date is:', data_dw['DATE'].max())
print('Min Date is:', data_dw['DATE'].min())

In [ ]:
# print(data_dw['MW'].count())
# print(data_dw[['DEPTH', 'DATE', 'LON', 'TYPE']])

In [ ]:
def Sumatra_map():
    
    fig, ax = plt.subplots(1, dpi=200)

    proj = ccrs.PlateCarree()                       # the projection you want to use, many are supported, 
                                                    # this one is the standard for geographic coordinates
    
    ax = plt.axes(projection=proj)                  # sets the projection

    
    ##### topo ##########
#     topo = ax.contourf(lon, lat, z, 30,
#         transform=proj, cmap='cmo.balance', alpha = 0.9) #tempo
    topo = ax.pcolormesh(lon, lat, z,
                         transform=proj, cmap='cmo.balance', vmin = -9000, vmax = 7000) #tempo

#     # # define topo colorbar
#     cbaxes = inset_axes(ax, width="40%", height="3.5%", loc=1,
#                        bbox_to_anchor=(0,0,1,1), bbox_transform=ax.transAxes)
#     cbar = plt.colorbar(topo,
#                  cax=cbaxes, ticks=[-5000,-2500, 0,2500, 5000], orientation='horizontal')

#     # define age colobar label
#     size=6
#     cbar.set_label(label='Topography (m)', size=size, weight='bold')
#     cbar.ax.tick_params(labelsize=size)        

    
#     ###### crustal age ##########
#     age = ax.pcolormesh(lon_age, lat_age, z_age,
#         transform=proj, cmap='cmo.dense', vmin = 0, vmax = 180) #tempo
    
#     # # define age colorbar
#     cbaxes = inset_axes(ax, width="30%", height="3.5%", loc=1,
#                        bbox_to_anchor=(0,0,1,1), bbox_transform=ax.transAxes)
#     cbar = plt.colorbar(age,
#                  cax=cbaxes, ticks=[0, 40, 80, 120, 160], orientation='horizontal')

#     # define age colobar label
#     size=6
#     cbar.set_label(label='Crustal Age (Ma)', size=size, weight='bold')
#     cbar.ax.tick_params(labelsize=size)    

    

    ####### slab depth data ######
    slab = ax.contourf(lon_slab, lat_slab, depth_slab*-1, 30,
        transform=proj, cmap='binary', alpha = 0.9)

    # define slab colorbar
    cbaxes = inset_axes(ax, width="3%", height="25%", loc=3,
                       bbox_to_anchor=(0,0,1,1), bbox_transform=ax.transAxes)
    cbar = plt.colorbar(slab,
                 cax=cbaxes, ticks=[0, 200, 400, 600], orientation='vertical')

    # define slab colobar label
    size=6
    cbar.set_label(label='Slab depth (Km)', size=size, weight='bold')
    cbar.ax.tick_params(labelsize=size)    

    
#     ax.set_extent([90, 115, -15, 15])
    
    ax.set_xlim(85,120)                             # set region for Banda Arc
    ax.set_ylim(-15,15)
#     ax.set_title("After 2004 GEq (upper plate)", fontsize=9)
    
#     ax.set_xlim(95,115)                             # set region for Banda Arc
#     ax.set_ylim(-12,0)
    

#     ax.add_feature(cartopy.feature.OCEAN            # add ocean feature
#                    , zorder=0                       # layer where ocean should plot; 0 is base   
#                    , facecolor='w'                  # color of the ocean
#                   )           

#     ax.add_feature(cartopy.feature.LAND             # add continent feature
#                    , zorder=0                       # layer where continent should plot; 0 is base
#                    , edgecolor='black'              # coastline color
#                    , facecolor='lightgray'          # continent color
#                   )               

    ax.coastlines(resolution='110m',
                  linewidth = 0.1)                # the level of resolution to render the drawings, 
                                                    # see documentation for more details
     
        
    gl = ax.gridlines(crs=ccrs.PlateCarree()        
                    , draw_labels=True              # label coordinates
                    , linewidth=0                   # line properties can be changed similar to matplotlib
                    , color='black'
                    , alpha=0.5
                    , linestyle='-')

    gl.xlabels_top = False                                           # where labels shouldn't be
    gl.ylabels_right = False
    gl.xlocator = mticker.FixedLocator(np.arange(85,120,5))          # sets longitude tick marks
    gl.ylocator = mticker.FixedLocator(np.arange(-15,15,5))          # sets latitude tick marks using numpy module
    gl.xformatter = LONGITUDE_FORMATTER                              # set cartopy formatting
    gl.yformatter = LATITUDE_FORMATTER

    gl.xlabel_style = {'size': 9}                                   # change label font sizes
    gl.ylabel_style = {'size': 9}
    
    
    
    ##plot the trench
    ax.plot(trench_xy['LON'], trench_xy['LAT'], '-', c = 'black', linewidth = 1, label = 'Trench')
#     ax.plot(sumatra_trench['LON'], sumatra_trench['LAT'], '-', c = 'tab:red', linewidth = 2, label = 'Trench')
#     ax.plot(java_trench['LON'], java_trench['LAT'], '-', c = 'tab:blue', linewidth = 2, label = 'Trench')
    

    ax.text(98, -12, 'Investigator FZ', transform=ccrs.PlateCarree(), color='black',
            fontsize=6, rotation=90, alpha=1,
            path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])

    ax.text(90, 0, 'NER', transform=ccrs.PlateCarree(), color='black',
              fontsize=6, rotation=86, alpha=1,
             path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])

    ax.text(101, -11, 'Christmas Island', transform=ccrs.PlateCarree(), color='black',
              fontsize=6, alpha=1,
             path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])

    ax.text(110.5, -13, 'Roo Rise', transform=ccrs.PlateCarree(), color='black',
              fontsize=6, alpha=1,
             path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])

    ax.text(92, -1.5, 'WB', transform=ccrs.PlateCarree(), color='black',
              fontsize=6, alpha=1,
             path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])    
    
    ax.text(98, -14.5, 'Indo-Australian Plate', transform=ccrs.PlateCarree(), ha='center',
            color='black', fontsize=7,
           path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])
    
    ax.text(110, 1, 'Sunda Plate', transform=ccrs.PlateCarree(),
            color='black', fontsize=7,
           path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])
    
    ax.text(94.5, 9, 'Andaman', transform=ccrs.PlateCarree(),
            color='black', fontsize=7,
           path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])

    ax.text(100.5, -2, 'Sumatra', transform=ccrs.PlateCarree(),
            color='black', fontsize=7, rotation=-50,
           path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])

    ax.text(109, -7, 'Java', transform=ccrs.PlateCarree(),
            color='black', fontsize=7,
           path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])
    
    ax.text(105, -6.5, 'Sunda\nStrait', transform=ccrs.PlateCarree(),
            color='black', fontsize=5, ha='center',
           path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])
    
#     ax.text(104, 6, 'Subducting plate after 2004 mainshock \nDepth < 650 km', transform=ccrs.PlateCarree(),
#                color='black', fontsize=6) #ha='center'
    
    
###plot rupture patch
    color_rup= 'springgreen'
    linewidth = 0.8
    ax.plot(rup_2004['LON'], rup_2004['LAT'], '-', c = color_rup, linewidth = linewidth, alpha=0.8)
    ax.plot(rup_2005['LON'], rup_2005['LAT'], '-', c = color_rup, linewidth = linewidth, alpha=0.8)
    ax.plot(rup_2007['LON'], rup_2007['LAT'], '-', c = color_rup, linewidth = linewidth, alpha=0.8)
    
# ## Plot Mw ≥ 8.5 megathrust as stars
#     ax.scatter(data_mega['LON'], data_mega['LAT'], marker = '*',
#                  s = 80, c = 'yellow', edgecolor = 'black',
#                  linewidths = 0.5, transform=cartopy.crs.PlateCarree())

#     rect = patches.Rectangle((93, -6), 14, 12, linewidth=0.5, linestyle='--',
#                          edgecolor='k', facecolor="none", label='Study area')
#     ax.add_patch(rect)
    
    return fig, ax, proj

In [ ]:
fig, ax, proj = Sumatra_map()           # make the basemap       

fig1 = ax

width=(0.6,0.6)

# # # ##### Strike Slip #########
for strike,dip,rake,lon,lat in zip(dataS['STRIKE.1'],
                                   dataS['DIP.1'],
                                   dataS['RAKE.1'],
                                   dataS['LON'],
                                   dataS['LAT']):

    

    bballs = beach([strike, dip, rake]   # build beachball as a collection
              , xy=(lon,lat)            # where to plot beachball in the map
              , width=width             # scale of the beachball in the (x,y) plane
              , linewidth=0.3             # thickness of nodal planes and beachball edge lines
              , facecolor='tab:brown'    # color of one of the axes
              , zorder=1
                , alpha = 0.6)               # make sure beachball plots on top

    fig1.add_collection(bballs)            # add the beachball the plot



###### Normal ######
for strike,dip,rake,lon,lat in zip(dataN['STRIKE.1'],
                                   dataN['DIP.1'],
                                   dataN['RAKE.1'],
                                   dataN['LON'],
                                   dataN['LAT']):



    bballn = beach([strike, dip, rake]   # build beachball as a collection
              , xy=(lon,lat)            # where to plot beachball in the map
              , width=width            # scale of the beachball in the (x,y) plane
              , linewidth=0.3             # thickness of nodal planes and beachball edge lines
              , facecolor='deepskyblue'    # color of one of the axes
              , zorder=1
                   , alpha = 0.9)               # make sure beachball plots on top

    fig1.add_collection(bballn)            # add the beachball the plot
    

###### Thrust ########
for strike,dip,rake,lon,lat in zip(dataT['STRIKE.1'],
                                   dataT['DIP.1'],
                                   dataT['RAKE.1'],
                                   dataT['LON'],
                                   dataT['LAT']):


    

    bballt = beach([strike, dip, rake]   # build beachball as a collection
              , xy=(lon,lat)            # where to plot beachball in the map
              , width=width            # scale of the beachball in the (x,y) plane
              , linewidth=0.3             # thickness of nodal planes and beachball edge lines
              , facecolor='lightcoral'    # color of one of the axes
              , zorder=1
                , alpha = 0.9)
#                   , zorder=dataT['YEARS'], cmap='seismic')               # make sure beachball plots on top

    fig1.add_collection(bballt)            # add the beachball the plot    


# # plot strike-slips as dots
# fig1.scatter(dataS['LON'], dataS['LAT'],
#            marker = 'o',
#            s = 5,
#            c = 'k',
#            alpha = 0.8,
#            edgecolor = 'white',
#            linewidths = 0.5,
#            transform=cartopy.crs.PlateCarree())
    


##add info of mega events
fig1.text(96, 4, 'Mw 9.0 (2004)', transform=ccrs.PlateCarree(), color='black', fontsize=5, fontstyle='italic', fontweight='bold',
        path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])
fig1.text(98.5, 3, 'Mw 8.6 (2005)', transform=ccrs.PlateCarree(), color='black', fontsize=5, fontstyle='italic', fontweight='bold',
        path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])
fig1.text(102, -4, 'Mw 8.5 (2007)', transform=ccrs.PlateCarree(), color='black', fontsize=5, fontstyle='italic', fontweight='bold',
        path_effects = [path_effects.Stroke(linewidth=1, foreground='white'), path_effects.Normal()])

## Plot Mw ≥ 8.5 megathrust as stars
fig1.scatter(data_mega['LON'], data_mega['LAT'], marker = '*',
           s = 60, c = 'springgreen', edgecolor = 'black',
           linewidths = 0.5, transform=cartopy.crs.PlateCarree())

x0, xmax = fig1.set_xlim()
y0, ymax = fig1.set_ylim()
data_width = xmax - x0
data_height = ymax - y0
# fig1.text(x0 + data_width*0.01, y0 + data_height * 0.03,
# #           '(a)',
#           '(b)',
#           fontsize=10, color='white')
# fig1.text(x0 + data_width*0.6, y0 + data_height * 0.9,
#           '1976-2004/12/25 \n (Depth < 300 km)',
# #           '2004/12/25-2009 \n (Depth < 300 km)',
#           fontsize=8, color='white', fontweight='bold')


plt.savefig("/Users/mohd0001/PhD_Monash/Paper2/output/map_upperplate.png",
            dpi = 300, transparent= True, bbox_inches='tight')

In [ ]:
# import matplotlib.pyplot as plt
# from obspy.imaging.beachball import beach

# fs=8
# alpha=0.7
# width = 25
# linewidth=0.3
#     #strike dip rake
# np1 = [0, 45, 90] #thrust
# mt = [0, 45, 270] #normal
# mn = [90, 90, 1] # strike-slip
# beach1 = beach(np1, xy=(0, 80), width=width, facecolor='tab:red', linewidth=linewidth)
# beach2 = beach(mt, xy=(40, 80), width=width, facecolor='blue', linewidth=linewidth)
# beach3 = beach(mn, xy=(80, 80), width=width, facecolor='tab:brown', linewidth=linewidth)

# fig = plt.subplots(1, dpi=200, figsize=(5,3))
# plt.plot([-60, 150], [0, 100], '.', c='white') 

# plt.text(-12, 60, 'Thrust', color='black', fontsize=fs)
# plt.text(27, 60, 'Normal', color='black', fontsize=fs)
# plt.text(62, 60, 'Strike-slip', color='black', fontsize=fs)

# plt.plot(-10, 50, "*",
#                 markersize = 12,
#                 linewidth=2,
#                 color = 'k',
#                 alpha=0.6,
#                 label='Normal (10 yrs after)',
#          mfc = 'yellow', mew = 0.8, lw = 0.8)
# plt.text(0, 48, 'Megathrust events ($M_w$ ≥ 8.5)', color='black', fontsize=fs)



# # plt.text(-55, -25, r'Data: CMT Catalog (1994-2014)', color='black', fontsize=fs)
# plt.text(-10, 35, r'$M_w$ < 5.5', color='black', fontsize=fs)
# plt.text(-10, 25, r'Depth < 50 km', color='black', fontsize=fs)
# # plt.text(-55, -55, r'Region: Andaman-Sunda Upper Plate', color='black', fontsize=fs)
# ax = plt.gca()
# ax.add_collection(beach1) 
# ax.add_collection(beach2) 
# ax.add_collection(beach3) 
# ax.set_aspect("equal")

# ax.set_axis_off()

# # ax.set_xlim((-120, 120))  
# # (-120, 120)
# # ax.set_ylim((-20, 120))  
# # (-20, 120)
# plt.savefig('FM_legendDW.jpg', dpi = 600, transparent= True, bbox_inches='tight')